In [1]:
import os

Variaveis globais necessárias para funçoes

In [2]:
datasetTxt = "Datasets/Dataset-ClassTT_02_real_File.txt"


Parse do dataset de horários, retornando um dicionário com todas as seções.

    Retorna:
        dict: Dicionário com as seguintes chaves:
            - 'head': dict com configurações gerais
            - 'cc': dict {turma: [cursos]}
            - 'olw': list de cursos com apenas 1 aula/semana
            - 'dsd': dict {professor: [cursos]}
            - 'tr': dict {professor: [slots indisponíveis]}
            - 'rr': dict {curso: sala}
            - 'oc': dict {curso: índice_aula_online}

In [3]:
# Cell 3: Importações e funções auxiliares
from constraint import Problem
import itertools

def slot_day(slot):
    """Converte slot (1-20) para dia da semana (0=Segunda, 1=Terça, 2=Quarta, 3=Quinta, 4=Sexta)"""
    return (slot - 1) // 4

def slot_time(slot):
    """Converte slot (1-20) para período do dia (0=manhã1, 1=manhã2, 2=tarde1, 3=tarde2)"""
    return (slot - 1) % 4

def day_name(day_index):
    """Retorna o nome do dia"""
    days = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]
    return days[day_index]

def time_name(time_index):
    """Retorna o nome do período"""
    times = ["9h-11h", "11h-13h", "14h-16h", "16h-18h"]
    return times[time_index]

def are_consecutive(slot1, slot2):
    """Verifica se dois slots são consecutivos no mesmo dia"""
    return slot_day(slot1) == slot_day(slot2) and abs(slot_time(slot1) - slot_time(slot2)) == 1

print("Funções auxiliares definidas")

Funções auxiliares definidas


In [4]:
def parse_dataset(datasetTxt):
    lines = datasetTxt.strip().split('\n')
    data = {
        'head': {},
        'cc': {},
        'olw': [],
        'dsd': {},
        'tr': {},
        'rr': {},
        'oc': {},
        'rooms': [],
    }

    current_section = None

    for line in lines:
        line = line.strip()

        # Ignorar linhas vazias
        if not line:
            continue

        # Detectar cabeçalhos de seção
        if line.startswith('#'):
            section_name = line.split()[0][1:]  # Remove o '#'
            current_section = section_name
            continue

        # Ignorar comentários
        if line.startswith('—') or line.startswith('--'):
            continue

        # Processar dados baseado na seção atual
        if current_section == 'cc':
            parts = line.split()
            if parts:
                class_id = parts[0]
                courses = parts[1:]
                data['cc'][class_id] = courses

        elif current_section == 'olw':
            # Esta seção parece estar vazia no exemplo
            if line.strip():
                data['olw'].append(line.strip())

        elif current_section == 'dsd':
            parts = line.split()
            if parts:
                teacher = parts[0]
                courses = parts[1:]
                data['dsd'][teacher] = courses

        elif current_section == 'tr':
            parts = line.split()
            if parts:
                teacher = parts[0]
                slots = [int(s) for s in parts[1:]]
                data['tr'][teacher] = slots

        elif current_section == 'rr':
            parts = line.split()
            if len(parts) >= 2:
                course = parts[0]
                room = parts[1]
                data['rr'][course] = room

        elif current_section == 'oc':
            parts = line.split()
            if len(parts) >= 2:
                course = parts[0]
                week_index = int(parts[1])
                data['oc'][course] = week_index

        elif current_section == 'rooms':
            if line.strip():
                data['rooms'].append(line.strip())

    return data

Lê o ficheiro do dataset.

In [5]:
def load_dataset_from_file(datasetTxt):
    try:
        with open(datasetTxt, 'r', encoding='utf-8') as f:
            dataset_text = f.read()
        return parse_dataset(dataset_text)
    except FileNotFoundError:
        print(f"Erro: Ficheiro '{datasetTxt}' não encontrado!")
        return None
    except Exception as e:
        print(f"Erro ao ler ficheiro: {e}")
        return None

print("✅ Parser do dataset definido")

✅ Parser do dataset definido


## Testes

In [6]:

# Carregar o dataset
dados = load_dataset_from_file(datasetTxt)

# Verificar se carregou
if dados is None:
    print("❌ Falha ao carregar o dataset!")
else:
    print("✅ Dataset carregado com sucesso!\n")

    # DEBUG: Ver o conteúdo de cada seção
    print("=" * 50)
    print("📊 CONTEÚDO PARSEADO:")
    print("=" * 50)

    print("\n🎓 HEAD (Configurações):")
    print(dados['head'])

    print("\n📚 CC (Classes e Cursos):")
    for turma, cursos in dados['cc'].items():
        print(f"  {turma}: {cursos}")

    print("\n👨‍🏫 DSD (Professores e Cursos):")
    for prof, cursos in dados['dsd'].items():
        print(f"  {prof}: {cursos}")

    print("\n⏰ TR (Restrições de Tempo):")
    for prof, slots in dados['tr'].items():
        print(f"  {prof}: slots {slots}")

    print("\n🚪 RR (Restrições de Sala):")
    for curso, sala in dados['rr'].items():
        print(f"  {curso}: {sala}")

    print("\n💻 OC (Aulas Online):")
    for curso, indice in dados['oc'].items():
        print(f"  {curso}: aula {indice}")

    print("\n📝 OLW (Cursos com 1 aula/semana):")
    print(f"  {dados['olw']}")

    print("\nROOMS:")
    if dados['rooms']:
        for sala in dados['rooms']:
            print(f"  - {sala}")

✅ Dataset carregado com sucesso!

📊 CONTEÚDO PARSEADO:

🎓 HEAD (Configurações):
{}

📚 CC (Classes e Cursos):
  LESI: ['PDM', 'ISI', 'IA', 'SETR', 'PA']
  LEEC: ['PS', 'IM', 'R', 'IE', 'RCSD']
  LEIM: ['GSI', 'IA', 'RCE', 'AST', 'ISC']
  LDJG: ['IAAJ', 'PVR', 'TCGEV', 'PAG', 'GUS']

👨‍🏫 DSD (Professores e Cursos):
  João: ['PDM', 'PS', 'IM', 'GSI']
  Pedro: ['ISI', 'R', 'IA']
  António: ['IA', 'SETR', 'IE', 'RCE']
  Manuel: ['PA', 'RCSD', 'AST', 'ISC', 'GUS']
  Ana: ['IAAJ', 'PAG']
  Isabel: ['PVR', 'TCGEV']

⏰ TR (Restrições de Tempo):
  Isabel: slots [13, 14, 15, 16, 17, 18, 19, 20]
  Manuel: slots [1, 2, 3, 4]
  João: slots [9, 10, 11, 12, 17, 18, 19, 20]

🚪 RR (Restrições de Sala):
  SETR: Lab01
  IM: Lab01

💻 OC (Aulas Online):
  PS: aula 2
  GSI: aula 2

📝 OLW (Cursos com 1 aula/semana):
  []

ROOMS:
  - SalaT
  - SalaC
  - Lab01


In [7]:
# ============================
# 1) VARIÁVEIS (duas por UC)
# ============================
lesson_vars = []                      # ex: "t01_UC11_L1"
lesson_meta = {}                      # var -> {"class":..., "course":..., "lesson":1|2}
for turma, ucs in dados['cc'].items():
    for uc in ucs:
        for i in (1, 2):              # cada UC tem 2 aulas/semana
            var = f"{turma}_{uc}_L{i}"
            lesson_vars.append(var)
            lesson_meta[var] = {"class": turma, "course": uc, "lesson": i}

print(f"Total de variáveis criadas: {len(lesson_vars)}")


Total de variáveis criadas: 40


In [ ]:
from constraint import Problem, MinConflictsSolver
from collections import defaultdict

problem = Problem(MinConflictsSolver())

# Pré-computar dias dos blocos
dia_do_bloco_cache = {}
for slot in range(1, 21):
    dia_do_bloco_cache[slot] = ((slot - 1) // 4) + 1

def dia_do_bloco(slot):
    return dia_do_bloco_cache[slot]

# Mapeamento curso->professor
curso_professor = {}
for prof, cursos in dados['dsd'].items():
    for c in cursos:
        curso_professor[c] = prof

# Lista de aulas online pré-computada
online_vars = []
for curso, li in dados['oc'].items():
    for turma, ucs in dados['cc'].items():
        if curso in ucs:
            online_vars.append(f"{turma}_{curso}_L{li}")

# Salas fixas pré-computadas
salas_fixas = {}
for curso, sala in dados['rr'].items():
    for turma, ucs in dados['cc'].items():
        if curso in ucs:
            for i in (1, 2):
                salas_fixas[f"{turma}_{curso}_L{i}"] = sala

# ===============================================
# VARIÁVEIS COM DOMÍNIOS JÁ FILTRADOS
# ===============================================

# Pré-computar disponibilidades por professor
professor_disponibilidade = {}
for prof in dados.get('tr', {}):
    professor_disponibilidade[prof] = [s for s in range(1, 21) if s not in dados['tr'][prof]]

for v in lesson_vars:
    uc = v.split("_")[1]
    prof = curso_professor.get(uc)
    
    if prof and prof in professor_disponibilidade:
        slots = professor_disponibilidade[prof]
    else:
        slots = list(range(1, 21))
    
    problem.addVariable(v, slots)

# ===============================================
# RESTRIÇÕES
# ===============================================

# 1. Aulas online após offline
def online_apos_offline(slot_online, slot_offline):
    return slot_online > slot_offline

for v_online in online_vars:
    turma, uc, _ = v_online.split("_")
    v_offline = f"{turma}_{uc}_L1"
    if v_offline in lesson_vars:
        problem.addConstraint(online_apos_offline, (v_online, v_offline))

# 2. Dias diferentes para L1 e L2 da mesma UC
def dias_diferentes(slot1, slot2):
    return dia_do_bloco(slot1) != dia_do_bloco(slot2)

for turma, ucs in dados['cc'].items():
    for uc in ucs:
        var1 = f"{turma}_{uc}_L1"
        var2 = f"{turma}_{uc}_L2"
        if var1 in lesson_vars and var2 in lesson_vars:
            problem.addConstraint(dias_diferentes, (var1, var2))

# 3. Professores não podem ter sobreposição
for prof, cursos in dados['dsd'].items():
    prof_vars = []
    for turma, ucs_turma in dados['cc'].items():
        for uc in ucs_turma:
            if uc in cursos:
                for i in (1, 2):
                    var = f"{turma}_{uc}_L{i}"
                    if var in lesson_vars:
                        prof_vars.append(var)
    
    if prof_vars:
        problem.addConstraint(lambda *args: len(args) == len(set(args)), prof_vars)

# 4. Turmas não podem ter sobreposição
for turma in dados['cc']:
    turma_vars = [v for v in lesson_vars if v.startswith(turma + "_")]
    if turma_vars:
        problem.addConstraint(lambda *args: len(args) == len(set(args)), turma_vars)

# 5. Máximo de aulas por dia por turma
def criar_restricao_max_aulas(turma_vars, max_por_dia=3):
    for dia in range(1, 6):
        blocos_este_dia = list(range((dia-1)*4 + 1, dia*4 + 1))
        
        def max_aulas_dia(*slots, blocos=blocos_este_dia, max_aulas=max_por_dia):
            count = 0
            for slot in slots:
                if slot in blocos:
                    count += 1
                    if count > max_aulas:
                        return False
            return True
        
        if turma_vars:
            problem.addConstraint(max_aulas_dia, turma_vars)

for turma in dados['cc']:
    turma_vars = [v for v in lesson_vars if v.startswith(turma + "_")]
    criar_restricao_max_aulas(turma_vars, 3)

# 6. Pelo menos um dia livre por turma
def pelo_menos_um_dia_livre(*slots):
    dias_com_aulas = set()
    for slot in slots:
        dia = dia_do_bloco(slot)
        dias_com_aulas.add(dia)
    
    return len(dias_com_aulas) < 5

for turma in dados['cc']:
    turma_vars = [v for v in lesson_vars if v.startswith(turma + "_")]
    if turma_vars:
        problem.addConstraint(pelo_menos_um_dia_livre, turma_vars)

# 7. Aulas da mesma UC em horários seguidos quando possível - SOFT
def aulas_seguidas_preferivel(slot1, slot2):
    dia1 = dia_do_bloco(slot1)
    dia2 = dia_do_bloco(slot2)
    
    # Se estão no mesmo dia, verifica se são horários seguidos
    if dia1 == dia2:
        if abs(slot1 - slot2) == 1:
            return True
        return True
    # Dias diferentes também é aceitável
    return True

for turma, ucs in dados['cc'].items():
    for uc in ucs:
        var1 = f"{turma}_{uc}_L1"
        var2 = f"{turma}_{uc}_L2"
        if var1 in lesson_vars and var2 in lesson_vars:
            problem.addConstraint(aulas_seguidas_preferivel, (var1, var2))

# ===============================================
# GESTÃO DE SALAS
# ===============================================

rooms_catalog = list(dados.get('rooms', []))
room_vars = {}

for var in lesson_vars:
    var_room = f"{var}_ROOM"
    room_vars[var] = var_room

    if var in online_vars:
        problem.addVariable(var_room, ["Online"])
    elif var in salas_fixas:
        problem.addVariable(var_room, [salas_fixas[var]])
    else:
        problem.addVariable(var_room, rooms_catalog)

# Restrição de salas
def no_room_conflict(slot_a, room_a, slot_b, room_b):
    # Aulas online nunca conflitam
    if room_a == "Online" or room_b == "Online":
        return True
    
    # Se são salas diferentes, não há conflito
    if room_a != room_b:
        return True
    
    # Se são mesma sala mas horários diferentes, não há conflito
    if slot_a != slot_b:
        return True
    
    # Conflito: mesma sala e mesmo horário
    return False

# Aplicar restrição a TODOS os pares de aulas
print("A aplicar restrições de sala...")
all_lesson_vars = list(lesson_vars)
conflict_count = 0

for i in range(len(all_lesson_vars)):
    for j in range(i + 1, len(all_lesson_vars)):
        va = all_lesson_vars[i]
        vb = all_lesson_vars[j]
        
        problem.addConstraint(
            no_room_conflict,
            (va, room_vars[va], vb, room_vars[vb])
        )
        conflict_count += 1

# ===============================================
# BUSCA DE SOLUÇÃO
# ===============================================

print("A buscar solução otimizada...")
solution = problem.getSolution()

if not solution:
    print("Nenhuma solução viável encontrada com todas as restrições.")
    print("A tentar sem a restrição de dia livre...")
    
    # Criar novo problema sem restrição de dia livre
    problem_simple = Problem(MinConflictsSolver())
    
    # Adicionar variáveis (mesmo código)
    for v in lesson_vars:
        uc = v.split("_")[1]
        prof = curso_professor.get(uc)
        if prof and prof in professor_disponibilidade:
            slots = professor_disponibilidade[prof]
        else:
            slots = list(range(1, 21))
        problem_simple.addVariable(v, slots)
        
    # 1. Aulas online após offline
    for v_online in online_vars:
        turma, uc, _ = v_online.split("_")
        v_offline = f"{turma}_{uc}_L1"
        if v_offline in lesson_vars:
            problem_simple.addConstraint(online_apos_offline, (v_online, v_offline))

    # 2. Dias diferentes para L1 e L2
    for turma, ucs in dados['cc'].items():
        for uc in ucs:
            var1 = f"{turma}_{uc}_L1"
            var2 = f"{turma}_{uc}_L2"
            if var1 in lesson_vars and var2 in lesson_vars:
                problem_simple.addConstraint(dias_diferentes, (var1, var2))

    # 3. Professores não podem ter sobreposição
    for prof, cursos in dados['dsd'].items():
        prof_vars = []
        for turma, ucs_turma in dados['cc'].items():
            for uc in ucs_turma:
                if uc in cursos:
                    for i in (1, 2):
                        var = f"{turma}_{uc}_L{i}"
                        if var in lesson_vars:
                            prof_vars.append(var)
        if prof_vars:
            problem_simple.addConstraint(lambda *args: len(args) == len(set(args)), prof_vars)

    # 4. Turmas não podem ter sobreposição
    for turma in dados['cc']:
        turma_vars = [v for v in lesson_vars if v.startswith(turma + "_")]
        if turma_vars:
            problem_simple.addConstraint(lambda *args: len(args) == len(set(args)), turma_vars)

    # 5. Máximo de aulas por dia
    for turma in dados['cc']:
        turma_vars = [v for v in lesson_vars if v.startswith(turma + "_")]
        criar_restricao_max_aulas(turma_vars, 3)

    # 6. Aulas seguidas (preferência)
    for turma, ucs in dados['cc'].items():
        for uc in ucs:
            var1 = f"{turma}_{uc}_L1"
            var2 = f"{turma}_{uc}_L2"
            if var1 in lesson_vars and var2 in lesson_vars:
                problem_simple.addConstraint(aulas_seguidas_preferivel, (var1, var2))

    # 7. Salas
    for var in lesson_vars:
        var_room = f"{var}_ROOM"
        if var in online_vars:
            problem_simple.addVariable(var_room, ["Online"])
        elif var in salas_fixas:
            problem_simple.addVariable(var_room, [salas_fixas[var]])
        else:
            problem_simple.addVariable(var_room, rooms_catalog)

    # Restrições de sala
    for i in range(len(all_lesson_vars)):
        for j in range(i + 1, len(all_lesson_vars)):
            va = all_lesson_vars[i]
            vb = all_lesson_vars[j]
            problem_simple.addConstraint(no_room_conflict, (va, room_vars[va], vb, room_vars[vb]))

    # Buscar solução simplificada
    solution = problem_simple.getSolution()
    
    if solution:
        print("Solução encontrada")

def bloco_para_horario(slot):
    dia_idx = (slot - 1) // 4
    bloco_dia = (slot - 1) % 4 + 1
    dias_semana = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]
    horas = {1: "09h–11h", 2: "11h–13h", 3: "14h–16h", 4: "16h–18h"}
    return dias_semana[dia_idx], horas[bloco_dia]

print("\n" + "="*60)
print("Horario final criado")

A aplicar restrições de sala...
A buscar solução otimizada...

Horario final criado


In [9]:
# ===============================================
# EXPORTAR PARA PDF
# ===============================================
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4, landscape
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet

if solution:
    print("\nA gerar PDF com horários...")

    pdf_path = "Horario_Geral.pdf"
    doc = SimpleDocTemplate(pdf_path, pagesize=landscape(A4))
    styles = getSampleStyleSheet()
    elements = []

    horas = ["09h–11h", "11h–13h", "14h–16h", "16h–18h"]
    dias_semana = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]

    for turma in sorted(dados['cc'].keys()):
        elements.append(Paragraph(f"Horário - Turma {turma}", styles['Heading1']))
        elements.append(Spacer(1, 12))

        horario = defaultdict(list)
        for v in lesson_vars:
            if v.startswith(turma + "_"):
                uc = v.split("_")[1]
                slot = solution[v]
                dia, hora = bloco_para_horario(slot)
                prof = curso_professor.get(uc, "—")
                sala = solution[room_vars[v]]
                horario[(dia, hora)].append((uc, prof, sala))

        tabela = [["Hora"] + dias_semana]
        for hora in horas:
            linha = [hora]
            for dia in dias_semana:
                celulas = [f"{uc}\n{prof}\n({sala})" for (uc, prof, sala) in horario.get((dia, hora), [])]
                linha.append("\n\n".join(celulas) if celulas else "")
            tabela.append(linha)

        t = Table(tabela, colWidths=[70] + [120]*5, rowHeights=50)
        t.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
            ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('FONTSIZE', (0, 0), (-1, -1), 8),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
            ('TOPPADDING', (0, 0), (-1, -1), 6),
        ]))

        elements.append(t)

        if turma != sorted(dados['cc'].keys())[-1]:
            elements.append(PageBreak())

    doc.build(elements)
    print(f"PDF criado com sucesso: {pdf_path}")


A gerar PDF com horários...
PDF criado com sucesso: Horario_Geral.pdf
